In [9]:
%load_ext autoreload
%autoreload 2

import os
import requests
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from tb.models.SimpleNet import SimpleNet
from tb.local.telegram_bot_login import BOT_TOKEN, CHAT_ID

In [ ]:
# Generate dummy classification data
X, y = make_classification(n_samples=1000, n_features=2, n_classes=2, n_redundant=0)
X = StandardScaler().fit_transform(X)  # normalize features

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Initialize lists to track metrics
test_accuracies, epochs = [], []

# Training loop
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            model.eval()
            test_preds = model(X_test_tensor).argmax(dim=1)
            acc = (test_preds == y_test_tensor).float().mean().item()
            print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Test Accuracy: {acc:.2%}")
            epochs.append(epoch+1)
            test_accuracies.append(acc)

# Plot the test accuracy
plt.figure(figsize=(5, 4))
sns.lineplot(x=epochs, y=test_accuracies, marker='o', label='Test Accuracy')
plt.title('Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.tight_layout()
img_path = os.path.join("assets", 'simple_training.png')
plt.savefig(img_path)
plt.show()

In [11]:
# Send the plot to the bot
url = f'https://api.telegram.org/bot{BOT_TOKEN}/sendPhoto'

with open(img_path, 'rb') as image_file:
    files = {'photo': image_file}
    data = {'chat_id': CHAT_ID, 'caption': "Run successful! Here is the test accuracy"}
    response = requests.post(url, files=files, data=data)


In [ ]:
# Not there is an error, and have the bot send you the stack trace
import traceback

x = 3

try:
    assert x == 4, "Run failed, x is not 4!"
except AssertionError as e:
    traceback.print_exc()
    url = f'https://api.telegram.org/bot{BOT_TOKEN}/sendMessage'
    data = {'chat_id': CHAT_ID, 'text': f"Run failed, x is not 4\n{traceback.format_exc()}"}
    response = requests.post(url, data=data)